# 🤖 OpenAI Agents SDK — Hands-On Tutorial

This notebook walks you through the **OpenAI Agents SDK** step by step:

| Section | What you'll learn |
|---|---|
| **1** | Installation & setup |
| **2** | Creating your first agent |
| **3** | Function calling (tools) |
| **4** | Multi-agent handoffs |
| **5** | Agents as tools (orchestrator pattern) |
| **6** | Full practical example — customer support system |
| **7** | Inspecting results & conversation history |
| **8** | Proposed project structure for production |

---

### Key concepts

The SDK has very few primitives:

- **Agent** — An LLM equipped with `instructions`, `tools`, and `handoffs`.
- **Runner** — Executes an agent loop: sends user input → LLM decides → calls tools / hands off → repeats until done.
- **Function tools** — Turn any Python function into a tool the LLM can call (schema is auto-generated).
- **Handoffs** — Let an agent transfer control to another agent.
- **Agents as tools** — Let a central "orchestrator" agent call sub-agents as tools without fully handing off control.

> **Docs:** https://openai.github.io/openai-agents-python/

---
## 1 · Installation & Setup

In [ ]:
# Install the SDK (only needs to run once)
%pip install openai-agents --quiet

In [ ]:
import os

# Set your OpenAI API key
# Option A: set it here directly (not recommended for production)
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Option B: load from a .env file (recommended)
# %pip install python-dotenv --quiet
# from dotenv import load_dotenv
# load_dotenv()

# Verify the key is set
assert os.environ.get("OPENAI_API_KEY"), "⚠️  Please set OPENAI_API_KEY before continuing."
print("✅ API key is set.")

---
## 2 · Your First Agent — Hello World

An `Agent` is simply an LLM with:
- **`name`** — for identification and tracing.
- **`instructions`** — the system prompt that tells the agent how to behave.
- **`model`** — which OpenAI model to use (defaults to the latest).

The `Runner` executes the agent loop.  
- `Runner.run_sync()` — blocking, synchronous call (easy for notebooks).
- `Runner.run()` — async, use with `await`.

In [ ]:
from agents import Agent, Runner

# Create a simple agent
simple_agent = Agent(
    name="Haiku Writer",
    instructions="You are a creative poet. When the user asks for something, respond with a haiku.",
    model="gpt-4o-mini",
)

# Run it synchronously
result = Runner.run_sync(simple_agent, "Write a haiku about Python programming.")

print("Agent:", result.final_output)

### Understanding the `RunResult`

The `result` object contains rich information about what happened:

In [ ]:
# The final text output from the agent
print("Final output:", result.final_output)
print()

# Which agent produced the final output (useful when handoffs happen)
print("Last agent:", result.last_agent.name)
print()

# All items generated during the run (messages, tool calls, handoffs, etc.)
print("Items generated:")
for item in result.new_items:
    print(f"  - {type(item).__name__}: {str(item)[:120]}")

---
## 3 · Function Calling (Tools)

The most powerful feature of agents: **function tools** let the LLM call your Python functions.

How it works:
1. You decorate a function with `@function_tool`.
2. The SDK auto-generates a JSON schema from the function signature + docstring.
3. The LLM sees the tool and can decide to call it.
4. The SDK executes your function and sends the result back to the LLM.
5. The LLM uses the result to form its final answer.

### 3.1 — Simple function tools

In [ ]:
from agents import Agent, Runner, function_tool
import json

# ---------- Define sample tools ----------

@function_tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.

    Args:
        city: The name of the city to check weather for.
    """
    # In real life, you'd call a weather API
    fake_weather = {
        "oslo": "Cloudy, 3°C, light rain",
        "paris": "Sunny, 18°C",
        "tokyo": "Humid, 28°C, partly cloudy",
        "new york": "Clear skies, 15°C",
    }
    return fake_weather.get(city.lower(), f"Weather data not available for {city}")


@function_tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression and return the result.

    Args:
        expression: A mathematical expression to evaluate, e.g. '2 + 2' or '100 * 1.25'.
    """
    try:
        result = eval(expression)  # simple demo — don't use eval in production!
        return str(result)
    except Exception as e:
        return f"Error: {e}"


# ---------- Create agent with tools ----------

tool_agent = Agent(
    name="Assistant",
    instructions=(
        "You are a helpful assistant. Use the tools available to you to answer the user's questions. "
        "Always prefer using a tool over making up information."
    ),
    tools=[get_weather, calculate],
    model="gpt-4o-mini",
)

# Try it out
result = Runner.run_sync(tool_agent, "What's the weather in Oslo? Also, what is 42 * 17?")
print(result.final_output)

### 3.2 — Inspecting tool calls

Let's look at what happened under the hood — we can see the LLM decided to call our tools:

In [ ]:
from agents.items import ToolCallItem, ToolCallOutputItem, MessageOutputItem

print("=== Conversation trace ===")
print()
for item in result.new_items:
    if isinstance(item, ToolCallItem):
        print(f"🔧 Tool call: {item.raw_item.name}({item.raw_item.arguments})")
    elif isinstance(item, ToolCallOutputItem):
        print(f"   ➜ Result: {item.output}")
    elif isinstance(item, MessageOutputItem):
        text = item.raw_item.content[0].text if item.raw_item.content else ""
        role = item.raw_item.role
        print(f"💬 [{role}]: {text[:200]}")
    print()

### 3.3 — Tools with complex types

You can use Pydantic models, TypedDicts, or any Python type as function arguments:

In [ ]:
from typing import Optional
from pydantic import BaseModel


class OrderLookup(BaseModel):
    order_id: str
    include_tracking: bool = False


# --- Sample data (pretend this is a database) ---
ORDERS_DB = {
    "ORD-001": {"status": "shipped", "item": "Laptop", "tracking": "TRACK-9876", "total": 1299.99},
    "ORD-002": {"status": "processing", "item": "Headphones", "tracking": None, "total": 249.50},
    "ORD-003": {"status": "delivered", "item": "Keyboard", "tracking": "TRACK-5432", "total": 89.99},
}


@function_tool
def lookup_order(order_id: str, include_tracking: bool = False) -> str:
    """Look up an order by its ID.

    Args:
        order_id: The order ID (e.g. ORD-001).
        include_tracking: Whether to include tracking information.
    """
    order = ORDERS_DB.get(order_id)
    if not order:
        return f"Order {order_id} not found."

    info = f"Order {order_id}: {order['item']} — Status: {order['status']} — Total: ${order['total']}"
    if include_tracking and order["tracking"]:
        info += f" — Tracking: {order['tracking']}"
    return info


@function_tool
def list_all_orders() -> str:
    """List all available orders in the system."""
    lines = []
    for oid, data in ORDERS_DB.items():
        lines.append(f"{oid}: {data['item']} ({data['status']})")
    return "\n".join(lines)


order_agent = Agent(
    name="Order Agent",
    instructions="You help users check their order status. Use the tools to look up orders.",
    tools=[lookup_order, list_all_orders],
    model="gpt-4o-mini",
)

result = Runner.run_sync(order_agent, "Can you show me all my orders, and then give me details on ORD-001 with tracking?")
print(result.final_output)

---
## 4 · Multi-Agent Handoffs

**Handoffs** let one agent transfer the entire conversation to another agent. This is the key pattern for building systems where different agents specialize in different tasks.

When a handoff occurs:
1. The LLM calls a tool named `transfer_to_<agent_name>`.
2. The new agent takes over and sees the full conversation history.
3. The new agent continues the conversation.

### 4.1 — Triage pattern (one router → specialized agents)

In [ ]:
from agents import Agent, Runner

# --- Specialized agents ---

billing_agent = Agent(
    name="Billing Agent",
    instructions=(
        "You are a billing specialist. You help users with invoices, payments, and billing questions. "
        "If the user asks about something outside billing, tell them you can only help with billing."
    ),
    model="gpt-4o-mini",
)

tech_support_agent = Agent(
    name="Tech Support Agent",
    instructions=(
        "You are a tech support specialist. You help users with technical issues, bugs, and setup problems. "
        "If the user asks about something outside tech support, tell them you can only help with technical issues."
    ),
    model="gpt-4o-mini",
)

sales_agent = Agent(
    name="Sales Agent",
    instructions=(
        "You are a sales specialist. You help users with product information, pricing, and purchases. "
        "If the user asks about something outside sales, tell them you can only help with sales."
    ),
    model="gpt-4o-mini",
)

# --- Triage agent that routes to the right specialist ---

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a customer support triage agent. Your job is to understand the user's request "
        "and hand off to the appropriate specialist:\n"
        "- Billing questions → Billing Agent\n"
        "- Technical issues → Tech Support Agent\n"
        "- Product/pricing questions → Sales Agent\n\n"
        "Ask a clarifying question if the intent is not clear."
    ),
    handoffs=[billing_agent, tech_support_agent, sales_agent],
    model="gpt-4o-mini",
)

# Test: user asks a billing question
result = Runner.run_sync(triage_agent, "I was charged twice for my last invoice. Can you help?")

print(f"Final agent: {result.last_agent.name}")
print(f"Response: {result.final_output}")

In [ ]:
# Test: user asks a tech question
result2 = Runner.run_sync(triage_agent, "My app keeps crashing when I try to upload a file.")

print(f"Final agent: {result2.last_agent.name}")
print(f"Response: {result2.final_output}")

### 4.2 — Handoffs with input data

You can require the LLM to provide structured data when handing off (e.g. a reason for escalation):

In [ ]:
from agents import Agent, Runner, handoff, RunContextWrapper
from pydantic import BaseModel


class EscalationData(BaseModel):
    reason: str
    priority: str  # "low", "medium", "high"


async def on_escalation(ctx: RunContextWrapper[None], input_data: EscalationData):
    # In production, you might log this, create a ticket, send a Slack message, etc.
    print(f"🚨 ESCALATION: reason={input_data.reason}, priority={input_data.priority}")


escalation_agent = Agent(
    name="Human Escalation Agent",
    instructions="You have received an escalated issue. Acknowledge the customer and let them know a human will follow up.",
    model="gpt-4o-mini",
)

frontline_agent = Agent(
    name="Frontline Agent",
    instructions=(
        "You are a frontline customer support agent. Try to answer the user's question. "
        "If the issue is too complex or the user is very frustrated, escalate to a human."
    ),
    handoffs=[
        handoff(
            agent=escalation_agent,
            on_handoff=on_escalation,
            input_type=EscalationData,
            tool_description_override="Escalate to a human agent when the issue is complex or the user is upset.",
        )
    ],
    model="gpt-4o-mini",
)

result = Runner.run_sync(
    frontline_agent,
    "I've been trying to resolve this billing error for 3 weeks now and nobody is helping me! I'm extremely frustrated!",
)

print(f"\nFinal agent: {result.last_agent.name}")
print(f"Response: {result.final_output}")

---
## 5 · Agents as Tools (Orchestrator Pattern)

Sometimes you don't want a full handoff (where the new agent takes over). Instead, you want a **central orchestrator** that *calls* sub-agents as tools and stays in control.

The difference:
- **Handoff**: Control is fully transferred; the new agent produces the final output.
- **Agent as tool**: The orchestrator calls the sub-agent, gets the result, and continues its own processing.

This is useful when you need to combine results from multiple agents.

In [ ]:
from agents import Agent, Runner

# --- Sub-agents (each is a specialist) ---

research_agent = Agent(
    name="Research Agent",
    instructions=(
        "You are a research assistant. When given a topic, provide a brief, factual summary "
        "with key points. Be concise (3-5 bullet points)."
    ),
    model="gpt-4o-mini",
)

writing_agent = Agent(
    name="Writing Agent",
    instructions=(
        "You are a professional writer. Take the provided information and write a short, "
        "engaging paragraph suitable for a blog post. Keep it under 100 words."
    ),
    model="gpt-4o-mini",
)

critic_agent = Agent(
    name="Critic Agent",
    instructions=(
        "You are an editorial critic. Review the provided text and give brief, constructive "
        "feedback on clarity, tone, and accuracy. Be specific."
    ),
    model="gpt-4o-mini",
)

# --- Orchestrator uses them as tools ---

orchestrator = Agent(
    name="Content Orchestrator",
    instructions=(
        "You are a content creation orchestrator. When a user asks for content on a topic:\n"
        "1. First, use the research tool to gather facts.\n"
        "2. Then, use the writing tool to create a blog paragraph from those facts.\n"
        "3. Finally, use the critic tool to review the paragraph.\n"
        "4. Present the final paragraph along with the critic's feedback."
    ),
    tools=[
        research_agent.as_tool(
            tool_name="research",
            tool_description="Research a topic and return key facts.",
        ),
        writing_agent.as_tool(
            tool_name="write_paragraph",
            tool_description="Write an engaging blog paragraph from provided information.",
        ),
        critic_agent.as_tool(
            tool_name="review_text",
            tool_description="Review and critique a piece of text.",
        ),
    ],
    model="gpt-4o-mini",
)

result = Runner.run_sync(
    orchestrator,
    "Create a short blog paragraph about the benefits of renewable energy.",
)

print(result.final_output)

---
## 6 · Full Practical Example — Customer Support System

Let's combine everything: **tools + handoffs + multiple agents** into a realistic customer support scenario with sample data.

### The scenario
A company has:
- A **triage agent** that routes customers.
- An **order agent** with tools to look up orders and process returns.
- A **billing agent** with tools to check account balance and apply credits.
- An **FAQ agent** for general questions.

In [ ]:
from agents import Agent, Runner, function_tool, handoff, RunContextWrapper
import json

# ============================
# SAMPLE DATA (our "database")
# ============================

CUSTOMERS = {
    "CUST-100": {"name": "Alice Johnson", "email": "alice@example.com", "balance": 150.00},
    "CUST-200": {"name": "Bob Smith", "email": "bob@example.com", "balance": 0.00},
    "CUST-300": {"name": "Carol Williams", "email": "carol@example.com", "balance": 45.50},
}

ORDERS = {
    "ORD-1001": {"customer": "CUST-100", "item": "Wireless Mouse", "price": 29.99, "status": "delivered"},
    "ORD-1002": {"customer": "CUST-100", "item": "USB-C Hub", "price": 49.99, "status": "shipped"},
    "ORD-1003": {"customer": "CUST-200", "item": "Mechanical Keyboard", "price": 89.99, "status": "processing"},
    "ORD-1004": {"customer": "CUST-300", "item": "Monitor Stand", "price": 45.50, "status": "delivered"},
}

FAQ_DATA = {
    "shipping": "We offer free shipping on orders over $50. Standard shipping takes 5-7 business days.",
    "returns": "You can return items within 30 days of delivery for a full refund.",
    "warranty": "All products come with a 1-year manufacturer warranty.",
    "hours": "Our support team is available Monday-Friday, 9 AM - 6 PM EST.",
}

print("✅ Sample data loaded.")

In [ ]:
# ============================
# TOOLS for the Order Agent
# ============================

@function_tool
def get_order_status(order_id: str) -> str:
    """Look up the status of an order.

    Args:
        order_id: The order ID, e.g. ORD-1001.
    """
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    return json.dumps(order, indent=2)


@function_tool
def get_customer_orders(customer_id: str) -> str:
    """Get all orders for a customer.

    Args:
        customer_id: The customer ID, e.g. CUST-100.
    """
    customer_orders = {oid: o for oid, o in ORDERS.items() if o["customer"] == customer_id}
    if not customer_orders:
        return f"No orders found for customer {customer_id}."
    return json.dumps(customer_orders, indent=2)


@function_tool
def initiate_return(order_id: str, reason: str) -> str:
    """Initiate a return for an order.

    Args:
        order_id: The order ID to return.
        reason: The reason for the return.
    """
    order = ORDERS.get(order_id)
    if not order:
        return f"Order {order_id} not found."
    if order["status"] != "delivered":
        return f"Cannot return order {order_id} — current status is '{order['status']}'. Only delivered orders can be returned."
    # In production, this would update the DB
    return f"✅ Return initiated for {order_id} ({order['item']}). Reason: {reason}. Refund of ${order['price']} will be processed within 5-7 business days."


# ============================
# TOOLS for the Billing Agent
# ============================

@function_tool
def get_account_balance(customer_id: str) -> str:
    """Check the account balance/credits for a customer.

    Args:
        customer_id: The customer ID.
    """
    customer = CUSTOMERS.get(customer_id)
    if not customer:
        return f"Customer {customer_id} not found."
    return f"Customer {customer['name']} ({customer_id}) has a balance of ${customer['balance']:.2f}"


@function_tool
def apply_credit(customer_id: str, amount: float, reason: str) -> str:
    """Apply a credit to a customer's account.

    Args:
        customer_id: The customer ID.
        amount: The credit amount to apply (positive number).
        reason: The reason for the credit.
    """
    customer = CUSTOMERS.get(customer_id)
    if not customer:
        return f"Customer {customer_id} not found."
    if amount <= 0:
        return "Credit amount must be positive."
    # In production, this would update the DB
    new_balance = customer["balance"] + amount
    return f"✅ Credit of ${amount:.2f} applied to {customer['name']}'s account. Reason: {reason}. New balance: ${new_balance:.2f}"


# ============================
# TOOL for the FAQ Agent
# ============================

@function_tool
def search_faq(topic: str) -> str:
    """Search the FAQ database for information on a topic.

    Args:
        topic: The topic to search for (e.g. 'shipping', 'returns', 'warranty').
    """
    # Simple keyword matching
    results = []
    for key, value in FAQ_DATA.items():
        if topic.lower() in key or key in topic.lower():
            results.append(f"{key.upper()}: {value}")
    if not results:
        return f"No FAQ entries found for '{topic}'. Available topics: {', '.join(FAQ_DATA.keys())}"
    return "\n".join(results)


print("✅ All tools defined.")

In [ ]:
# ============================
# BUILD THE AGENT SYSTEM
# ============================

# Specialist: Order Agent
order_agent = Agent(
    name="Order Agent",
    instructions=(
        "You are an order management specialist. Help customers check order status, "
        "view their orders, and process returns. Use your tools to look up information. "
        "The current customer is Alice Johnson (CUST-100). "
        "Always be helpful and professional."
    ),
    tools=[get_order_status, get_customer_orders, initiate_return],
    model="gpt-4o-mini",
)

# Specialist: Billing Agent
billing_agent = Agent(
    name="Billing Agent",
    instructions=(
        "You are a billing specialist. Help customers with account balance inquiries "
        "and apply credits when appropriate. Use your tools to look up information. "
        "The current customer is Alice Johnson (CUST-100). "
        "Always be helpful and professional."
    ),
    tools=[get_account_balance, apply_credit],
    model="gpt-4o-mini",
)

# Specialist: FAQ Agent
faq_agent = Agent(
    name="FAQ Agent",
    instructions=(
        "You are an FAQ specialist. Answer general questions about the company's policies. "
        "Use the FAQ search tool to find accurate information. Be friendly and concise."
    ),
    tools=[search_faq],
    model="gpt-4o-mini",
)

# Triage Agent (the entry point)
triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are the front-desk triage agent for customer support. "
        "Greet the customer warmly and determine what they need help with, then hand off to the right specialist:\n\n"
        "- Order status, returns, deliveries → Order Agent\n"
        "- Account balance, credits, billing → Billing Agent\n"
        "- General questions, policies, shipping info → FAQ Agent\n\n"
        "If the request is ambiguous, ask a clarifying question before handing off."
    ),
    handoffs=[order_agent, billing_agent, faq_agent],
    model="gpt-4o-mini",
)

print("✅ Agent system built.")
print(f"   Triage → can hand off to: {[a.name for a in [order_agent, billing_agent, faq_agent]]}")

In [ ]:
# --- Test 1: Order-related query ---
print("=" * 60)
print("TEST 1: Order query")
print("=" * 60)

result = Runner.run_sync(
    triage_agent,
    "Hi, I'd like to check the status of my orders and maybe return the wireless mouse.",
)

print(f"\n🤖 Final agent: {result.last_agent.name}")
print(f"\n💬 Response:\n{result.final_output}")

In [ ]:
# --- Test 2: Billing query ---
print("=" * 60)
print("TEST 2: Billing query")
print("=" * 60)

result2 = Runner.run_sync(
    triage_agent,
    "What's my current account balance? I'm customer CUST-100.",
)

print(f"\n🤖 Final agent: {result2.last_agent.name}")
print(f"\n💬 Response:\n{result2.final_output}")

In [ ]:
# --- Test 3: FAQ query ---
print("=" * 60)
print("TEST 3: FAQ query")
print("=" * 60)

result3 = Runner.run_sync(
    triage_agent,
    "What is your return policy and shipping times?",
)

print(f"\n🤖 Final agent: {result3.last_agent.name}")
print(f"\n💬 Response:\n{result3.final_output}")

---
## 7 · Inspecting Results & Conversation History

Understanding what happened during a run is crucial for debugging and improving your agents. Let's write a helper to visualize the full conversation trace:

In [ ]:
from agents.items import (
    MessageOutputItem,
    HandoffCallItem,
    HandoffOutputItem,
    ToolCallItem,
    ToolCallOutputItem,
)


def print_conversation_trace(run_result):
    """Pretty-print the full conversation trace from a RunResult."""
    print("\n" + "=" * 60)
    print("📋 CONVERSATION TRACE")
    print("=" * 60)

    for i, item in enumerate(run_result.new_items, 1):
        if isinstance(item, MessageOutputItem):
            role = item.raw_item.role
            text = ""
            if hasattr(item.raw_item, "content") and item.raw_item.content:
                for part in item.raw_item.content:
                    if hasattr(part, "text"):
                        text += part.text
            agent_name = item.agent.name if hasattr(item, "agent") else "unknown"
            print(f"\n  [{i}] 💬 Message ({role}) — Agent: {agent_name}")
            print(f"      {text[:300]}")

        elif isinstance(item, ToolCallItem):
            print(f"\n  [{i}] 🔧 Tool Call: {item.raw_item.name}")
            print(f"      Args: {item.raw_item.arguments}")

        elif isinstance(item, ToolCallOutputItem):
            print(f"  [{i}] 📤 Tool Result:")
            print(f"      {str(item.output)[:300]}")

        elif isinstance(item, HandoffCallItem):
            print(f"\n  [{i}] 🔀 Handoff Call: {item.raw_item.name}")

        elif isinstance(item, HandoffOutputItem):
            target = item.target_agent.name if hasattr(item, "target_agent") else "unknown"
            source = item.source_agent.name if hasattr(item, "source_agent") else "unknown"
            print(f"  [{i}] 🔀 Handoff: {source} → {target}")

        else:
            print(f"\n  [{i}] ❓ {type(item).__name__}: {str(item)[:200]}")

    print("\n" + "=" * 60)
    print(f"✅ Final agent: {run_result.last_agent.name}")
    print(f"✅ Final output: {run_result.final_output[:300]}")
    print("=" * 60)


# Use it on our earlier result
print_conversation_trace(result)

---
## 8 · Proposed Project Structure for Production

When moving beyond a notebook into a real project, here's how you should structure a multi-agent system:

```
my-agent-project/
│
├── .env                          # API keys (OPENAI_API_KEY, etc.) — never commit this!
├── .env.example                  # Template showing required env vars
├── .gitignore                    # Ignore .env, __pycache__, .venv, etc.
├── pyproject.toml                # Project metadata & dependencies
├── README.md                     # Project documentation
│
├── src/
│   ├── __init__.py
│   │
│   ├── agents/                   # Agent definitions — one file per agent (or group)
│   │   ├── __init__.py
│   │   ├── triage.py             # Triage / router agent
│   │   ├── order_agent.py        # Order management specialist
│   │   ├── billing_agent.py      # Billing specialist
│   │   └── faq_agent.py          # FAQ / general questions
│   │
│   ├── tools/                    # Tool functions — grouped by domain
│   │   ├── __init__.py
│   │   ├── order_tools.py        # get_order_status, initiate_return, etc.
│   │   ├── billing_tools.py      # get_account_balance, apply_credit, etc.
│   │   └── faq_tools.py          # search_faq, etc.
│   │
│   ├── models/                   # Pydantic models for structured data
│   │   ├── __init__.py
│   │   ├── customer.py           # Customer, EscalationData, etc.
│   │   └── order.py              # Order, ReturnRequest, etc.
│   │
│   ├── services/                 # External integrations (DB, APIs, etc.)
│   │   ├── __init__.py
│   │   ├── database.py           # Database connection and queries
│   │   └── external_api.py       # External API clients
│   │
│   ├── config.py                 # App configuration, load .env, model settings
│   └── main.py                   # Entry point — build the agent graph and run
│
├── notebooks/                    # Jupyter notebooks for experimentation
│   └── exploration.ipynb
│
└── tests/                        # Tests for tools, agents, and flows
    ├── __init__.py
    ├── test_tools.py             # Unit tests for tool functions
    ├── test_agents.py            # Integration tests for agent behavior
    └── test_flows.py             # End-to-end conversation flow tests
```

### Key design principles

| Principle | Why |
|---|---|
| **Separate agents from tools** | Agents are *configuration* (instructions, model, handoffs). Tools are *logic*. Keep them apart so you can test tools independently. |
| **One agent per file** | Makes it easy to find, update, and version-control each agent's prompt and behavior. |
| **Group tools by domain** | `order_tools.py`, `billing_tools.py` — each file contains related tools. Easier to maintain. |
| **Pydantic models in `models/`** | Define your data shapes once, reuse in tools and handoff inputs. |
| **Services layer** | Real database queries and API calls go here. Tools call services, not the other way around. |
| **Config in one place** | Load `.env`, set model names, default temperatures, etc. in `config.py`. |
| **Test your tools** | Tool functions are pure Python — easy to unit test. Test that `get_order_status("ORD-001")` returns the right thing. |

### Example: what `src/agents/triage.py` would look like

```python
# src/agents/triage.py
from agents import Agent
from src.agents.order_agent import order_agent
from src.agents.billing_agent import billing_agent
from src.agents.faq_agent import faq_agent
from src.config import DEFAULT_MODEL

triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are the front-desk triage agent. Route customers to:\n"
        "- Order Agent for order/return questions\n"
        "- Billing Agent for balance/credit questions\n"
        "- FAQ Agent for general policy questions\n"
    ),
    handoffs=[order_agent, billing_agent, faq_agent],
    model=DEFAULT_MODEL,
)
```

### Example: what `src/tools/order_tools.py` would look like

```python
# src/tools/order_tools.py
from agents import function_tool
from src.services.database import get_order, get_orders_for_customer

@function_tool
def get_order_status(order_id: str) -> str:
    """Look up the status of an order.

    Args:
        order_id: The order ID, e.g. ORD-1001.
    """
    order = get_order(order_id)
    if not order:
        return f"Order {order_id} not found."
    return order.to_summary_string()
```

### Example: what `src/main.py` would look like

```python
# src/main.py
import asyncio
from agents import Runner
from src.agents.triage import triage_agent
from src.config import load_config

async def main():
    load_config()  # loads .env, sets up tracing, etc.

    result = await Runner.run(
        triage_agent,
        input="I'd like to check on my order ORD-1001.",
    )
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())
```

---

## Summary & Cheatsheet

| What | How |
|---|---|
| Create an agent | `Agent(name=..., instructions=..., model=..., tools=[...], handoffs=[...])` |
| Run synchronously | `Runner.run_sync(agent, "user message")` |
| Run async | `await Runner.run(agent, "user message")` |
| Define a tool | `@function_tool` decorator on any Python function |
| Handoff to another agent | Add agents to `handoffs=[agent_a, agent_b]` |
| Customize a handoff | `handoff(agent=..., on_handoff=..., input_type=...)` |
| Use agent as a tool | `agent.as_tool(tool_name=..., tool_description=...)` |
| Get final output | `result.final_output` |
| See which agent finished | `result.last_agent.name` |
| Inspect conversation | Loop over `result.new_items` |

### Next steps
- 📖 [Full documentation](https://openai.github.io/openai-agents-python/)
- 🔍 [Tracing & debugging](https://openai.github.io/openai-agents-python/tracing/)
- 🛡️ [Guardrails](https://openai.github.io/openai-agents-python/guardrails/)
- 🔊 [Voice / Realtime agents](https://openai.github.io/openai-agents-python/realtime/quickstart/)
- 🧩 [MCP server integration](https://openai.github.io/openai-agents-python/mcp/)